In [ ]:
!rm -rf /content/sample_data
!nvidia-smi 

Tue Feb 23 13:30:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U git+https://github.com/qubvel/segmentation_models.pytorch
!pip install -U albumentations
!pip install ttach
!pip install pytorch_toolbelt

In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
import cv2
import time
from io import BytesIO
import base64
from PIL import Image

from tqdm import tqdm
import glob
import os
from scipy.io import loadmat

import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn as nn
from torch.cuda.amp import autocast
import ttach as tta
from tqdm import tqdm_notebook

In [ ]:
Image.MAX_IMAGE_PIXELS = 1000000000000000
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
def get_infer_transform():
    transform = A.Compose([
        A.Resize(256, 256),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])
    return transform

def inference(model, img_dir):
    transform=get_infer_transform()
    image = cv2.imread(img_dir, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img = transform(image=image)['image']
    img=img.unsqueeze(0)
 
    with torch.no_grad():
        img=img.cuda()
        model = model
        output = model(img)
    
    pred = output.squeeze().cpu().data.numpy()
    pred = np.argmax(pred,axis=0)
    return pred+1

In [ ]:
!cp -r /content/drive/Shareddrives/ACloudDrive/Tianchi/2021全国数字生态创新大赛-智能算法赛/suichang_round1_test_partA_210120.zip ./
!unzip -q suichang_round1_test_partA_210120.zip

replace suichang_round1_test_partA_210120/000001.tif? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!rm -rf results
!mkdir results

In [ ]:
model_name = 'efficientnet-b6'
n_class=10
model = smp.UnetPlusPlus(
    encoder_name="efficientnet-b6",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights='imagenet',  # use `imagenet` pretreined weights for encoder initialization
    in_channels=3,  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=10,  # model output channels (number of classes in your dataset)
)
model.to(DEVICE)
model.load_state_dict(torch.load("/content/drive/Shareddrives/ACloudDrive/Tianchi/2021全国数字生态创新大赛-智能算法赛/model/upp_efb3_a_11.pth"))
model.eval()
tta_model = tta.SegmentationTTAWrapper(model, tta.aliases.d4_transform(), merge_mode='mean')

assert_list=[1,2,3,4,5,6,7,8,9,10]

out_dir='results/'
if not os.path.exists(out_dir):os.makedirs(out_dir)
test_paths=glob.glob('suichang_round1_test_partA_210120/*')
for per_path in tqdm_notebook(test_paths):
    result=inference(tta_model, per_path)
    img=Image.fromarray(np.uint8(result))
    img=img.convert('L')
    out_path=os.path.join(out_dir,per_path.split('/')[-1][:-4]+'.png')
    img.save(out_path)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b6-c76e70fd.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b6-c76e70fd.pth


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
!zip -r -q results.zip results/